<a href="https://colab.research.google.com/github/GOLISHYAMP/Colab_Notebooks/blob/main/HyperparametertunningANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikeras

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data = pd.read_csv('/content/Churn_Modelling.csv')

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
X = df.drop(['Exited'], axis = 1)
y = df['Exited']
LE = LabelEncoder()
X['Gender'] = LE.fit_transform(X['Gender'])
OHE = OneHotEncoder(drop='first')
ct = ColumnTransformer([
    ('OneHotEncoder', OHE, ['Geography'])
], remainder='passthrough')
X = ct.fit_transform(X)
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 31)
with open('LabelEncoder.pkl', 'wb') as f:
  pickle.dump(LE, f)
  f.close()
with open('ColumnTransformer.pkl', 'wb') as f:
  pickle.dump(ct, f)
  f.close()
with open('StandardScaler.pkl', 'wb') as f:
  pickle.dump(sc, f)
  f.close()

In [6]:
(X_train.shape[1],)

(11,)

In [7]:
def create_model(neurons = 32, layers = 1):
  model = Sequential()
  model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
  for _ in range(layers-1):
    model.add(Dense(neurons, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model


In [8]:
# Create keras classifier
model = KerasClassifier(neurons = 32, layers = 1, build_fn=create_model, verbose = 1)
param_grid = {
    'neurons' : [16, 32, 64, 128],
    'layers' : [1,2],
    'epochs' : [50, 100]
}

In [9]:
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT available")


GPU is available


In [10]:
grid= GridSearchCV(estimator=model, param_grid=param_grid, n_jobs = -1, cv = 3, verbose = 1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s"% (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7614 - loss: 0.5238
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8147 - loss: 0.4269
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8386 - loss: 0.4023
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8363 - loss: 0.3974
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8475 - loss: 0.3728
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8526 - loss: 0.3599
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8636 - loss: 0.3513
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8598 - loss: 0.3522
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8581 - loss: 0.3459
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8537 - loss: 0.3526
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8612 - loss: 0.3407
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [12]:
X_test[0]

array([-0.57873591,  1.74273971,  0.88435134, -1.09598752, -1.04143285,
       -1.04175968, -1.22584767,  0.80773656,  0.64609167,  0.97024255,
        0.30154023])

In [17]:
result = grid.best_estimator_.predict(X_test[0].reshape(1,-1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [20]:
np.round(result[0])

0